<a href="https://colab.research.google.com/github/yamaguchinorimitsu/ajax_app/blob/master/%E7%B7%B4%E7%BF%92%EF%BC%91%EF%BC%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, accuracy_score
import numpy as np

# CSVファイルを読み込み
df = pd.read_csv('/content/cinema.csv')

df2 = df.drop('cinema_id',axis=1)
df2.head()

,SNS1,SNS2,actor,original,sales
0,291.0,1044,8808.994029,0,9731
1,363.0,568,10290.709370,1,10210
2,158.0,431,6340.388534,1,8227
3,261.0,578,8250.485081,0,9658
4,209.0,683,10908.539550,0,9286


In [32]:

# 欠損値を平均値で補完
df3 = df2.fillna(df.mean())


In [54]:
from sklearn.preprocessing import StandardScaler

# StandardScalerのインスタンスを作成 そして標準化
sc = StandardScaler()
sc_df3 = sc.fit_transform(df3)


,SNS1,SNS2,actor,original,sales
0,291.0,1044,8808.994029,0,9731
1,363.0,568,10290.709370,1,10210
2,158.0,431,6340.388534,1,8227
3,261.0,578,8250.485081,0,9658
4,209.0,683,10908.539550,0,9286
...,...,...,...,...,...
95,494.0,1050,11137.482810,1,10537
96,505.0,928,11376.038540,1,10084
97,368.0,966,10393.252480,0,10069
98,326.0,1068,9454.019853,1,10218


In [34]:
from sklearn.decomposition import PCA


# PCAのインスタンスを作成し、主成分数(2か３が多いが次元の数まで作成できる)
# 白色化の設定を指定
model = PCA(whiten=True)



In [57]:
model.fit_transform(sc_df3)

model.explained_variance_ratio_


array([0.54012068, 0.1860554 , 0.13631136])

In [51]:
# 寄与率の合計を保持する変数
cumulative_ratio = 0.0

# 新規の列の数をカウントする変数
num_columns = 0

# 寄与率のリスト
variance_ratio = model.explained_variance_ratio_

# forループで累積の寄与率を計算
for ratio in variance_ratio:
    cumulative_ratio += ratio
    num_columns += 1
    if cumulative_ratio >= 0.8:
        break

# 結果を表示
print("新規の列の数 n=:", num_columns)

新規の列の数 n=: 3


In [56]:
model = PCA(n_components=3, whiten=True)

# 学習
model.fit(sc_df3)

# 学習済みの主成分分析モデルで新規データを変換
new = pd.DataFrame(model.transform(sc_df3), columns=['PC1', 'PC2', 'PC3'])

new_df3 = pd.concat([new, pd.DataFrame(df3)], axis=1)

cor_df = new_df3.corr()

cor_df.loc['PC1':'PC3', 'SNS1':]




,SNS1,SNS2,actor,original,sales
PC1,0.736368,0.598555,0.837999,0.469117,0.936901
PC2,-0.050756,-0.534089,-0.037073,0.800559,0.013415
PC3,-0.531509,0.536704,-0.017275,0.324773,-0.072303


In [62]:

# 目的変数と説明変数を定義
target = df3['sales']
features = new_df3[['PC1', 'PC2', 'PC3']]

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# 回帰モデルを選択
regressor = LinearRegression()

# モデルを訓練
regressor.fit(X_train, y_train)

# テストデータを使用して予測
y_pred = regressor.predict(X_test)

# 回帰モデルの性能を評価
mse = mean_squared_error(y_test, y_pred)

print("Mean Squared Error:", mse)

Mean Squared Error: 84807.61320707462


In [63]:
# 係数と切片の取得
coefficients = regressor.coef_
intercept = regressor.intercept_

# 結果の表示
print("係数:", coefficients)
print("切片:", intercept)

係数: [743.0940446   -6.88470755 -45.48908638]
切片: 9876.357669558649
